In [ ]:
!git clone https://github.com/cd1m0/evm2llvm.git

Cloning into 'evm2llvm'...
remote: Enumerating objects: 14, done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 14
Unpacking objects: 100% (14/14), 2.07 KiB | 530.00 KiB/s, done.


In [ ]:
!wget "http://archive.ubuntu.com/ubuntu/pool/main/libf/libffi/libffi6_3.2.1-8_amd64.deb"
!sudo dpkg -i libffi6_3.2.1-8_amd64.deb

--2023-01-23 11:21:22--  http://archive.ubuntu.com/ubuntu/pool/main/libf/libffi/libffi6_3.2.1-8_amd64.deb
Resolving archive.ubuntu.com (archive.ubuntu.com)... 91.189.91.39, 185.125.190.39, 185.125.190.36, ...
Connecting to archive.ubuntu.com (archive.ubuntu.com)|91.189.91.39|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17948 (18K) [application/x-debian-package]
Saving to: ‘libffi6_3.2.1-8_amd64.deb’

libffi6_3.2.1-8_amd 100%[===================>]  17.53K  --.-KB/s    in 0.04s   

2023-01-23 11:21:22 (426 KB/s) - ‘libffi6_3.2.1-8_amd64.deb’ saved [17948/17948]



In [ ]:
!wget "http://archive.ubuntu.com/ubuntu/pool/universe/l/llvm-toolchain-5.0/libllvm5.0_5.0.1-4_amd64.deb"
!sudo dpkg -i libllvm5.0_5.0.1-4_amd64.deb

In [ ]:
!sudo apt install apt-file
!sudo apt-file update
!apt-file search llvm-objdump

In [ ]:
!printf "deb http://apt.llvm.org/xenial/ llvm-toolchain-xenial-5.0 main" |sudo tee /etc/apt/sources.list.d/llvm-toolchain-xenial-5.0.list
!wget -O - https://apt.llvm.org/llvm-snapshot.gpg.key |sudo apt-key add -
!sudo apt update
!sudo apt install -y llvm-5.0 llvm-5.0-dev llvm-5.0-tools

In [ ]:
CC=g++
CFLAGS=-g -I./evmjit/libevmjit -I./evmjit/include -I./evmjit/evmc/include -I/content/evm2llvm/evmjit/include -I/content/evm2llvm/evmjit/deps/include
LDFLAGS=-L./evmjit/build/libevmjit/ -levmjit-standalone-thin -lpthread -I/content/evm2llvm/evmjit/include -I/content/evm2llvm/evmjit/deps/include

%.o:	%.cc
	$(CC) -c -o $@ $< $(CFLAGS)

evmjit/build/libevmjit/libevmjit-standalone-thin.a:
	[ -d evmjit/build ] || mkdir evmjit/build
	cd evmjit/build && cmake ..
	cd evmjit/build && cmake --build . --config RelWithDebInfo


libevmjit:	evmjit/build/libevmjit/libevmjit-standalone-thin.a

lift:	lift.o libevmjit
	$(CC) -o $@ lift.o $(LDFLAGS)

all:	lift

clean:
	rm -f lift *.o

In [ ]:
!/usr/lib/llvm-5.0/bin/opt -O3 -S /content/test.ll -o O3.ll

In [ ]:
!cd evm2llvm && git submodule set-url /content/evm2llvm/evmjit https://github.com/ethereum/evmjit.git

In [ ]:
!cd evm2llvm && git submodule update --remote --merge && git submodule update --init --recursive

In [ ]:
!cd evm2llvm && make

In [ ]:
!cd evm2llvm && make all

In [ ]:
!/content/evm2llvm/lift /content/test.bin > test.ll

In [ ]:
!git clone https://github.com/ethereum/aleth.git

In [ ]:
!cd aleth && git fetch origin && git reset --hard 168093d0eca9f364852f2c235cfe7300f5e8f4f4 && git submodule update --init

HEAD is now at 168093d0e Improve gitignore of jetbrains stuff
Submodule path 'cmake/cable': checked out 'd4de52118b0947e22a2bc93ba9be148da82fc6c5'
Submodule path 'evmc': checked out '354ba6f540655649e081fb455e21998186bf5576'
Submodule path 'test/jsontests': checked out 'a69eb762fc5665814ac21cf091eff3bd0bf4ef27'


In [ ]:
!sudo apt-get install libleveldb-dev

In [ ]:
# cryptopp has very bad CMakeLists.txt config.
# We have to enforce "cross compiling mode" there by setting CMAKE_SYSTEM_VERSION=NO
# to any "false" value.
hunter_config(cryptopp VERSION ${HUNTER_cryptopp_VERSION} CMAKE_ARGS CMAKE_SYSTEM_VERSION=NO)

hunter_config(
    libjson-rpc-cpp
    VERSION ${HUNTER_libjson-rpc-cpp_VERSION}
    CMAKE_ARGS
    UNIX_DOMAIN_SOCKET_SERVER=NO
    UNIX_DOMAIN_SOCKET_CLIENT=NO
    FILE_DESCRIPTOR_SERVER=NO
    FILE_DESCRIPTOR_CLIENT=NO
    TCP_SOCKET_SERVER=NO
    TCP_SOCKET_CLIENT=NO
    HTTP_SERVER=NO
    HTTP_CLIENT=NO
)

hunter_config(
    Boost
    URL "https://boostorg.jfrog.io/artifactory/main/release/1.67.0/source/boost_1_67_0.tar.gz"
    SHA1 77e73c9fd7bf85b14067767b9e8fdc39b49ee0f2
)

hunter_config(ethash VERSION 0.4.0
    URL https://github.com/chfast/ethash/archive/v0.4.0.tar.gz
    SHA1 fd92ffadff2931877a3b68685dd8c53f0bdec539
)

In [ ]:
!rm -rf /root/.hunter/

In [ ]:
!cd aleth && cd build && CXXFLAGS="-Wno-error=deprecated-copy" cmake -DEVMJIT=on -DCMAKE_CXX_FLAGS="-Wno-error=deprecated-copy" .. && cmake --build .

In [ ]:
!sudo rm -rf /root/.hunter/_Base/Download/crc32c

In [ ]:
!wget "https://github.com/hunter-packages/crc32c/archive/refs/tags/hunter-1.0.5.tar.gz" -O /root/.hunter/_Base/Download/crc32c/1.0.5/dc7fa8c/hunter-1.0.5.tar.gz

--2023-01-23 17:50:46--  https://github.com/hunter-packages/crc32c/archive/refs/tags/hunter-1.0.5.tar.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/hunter-packages/crc32c/tar.gz/refs/tags/hunter-1.0.5 [following]
--2023-01-23 17:50:46--  https://codeload.github.com/hunter-packages/crc32c/tar.gz/refs/tags/hunter-1.0.5
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘/root/.hunter/_Base/Download/crc32c/1.0.5/dc7fa8c/hunter-1.0.5.tar.gz’

/root/.hunter/_Base     [ <=>                ]  27.91K  --.-KB/s    in 0.02s   

2023-01-23 17:50:47 (1.10 MB/s) - ‘/root/.hunter/_Base/Download/crc32c/1.0.5/dc7fa8c/hunter-1.0.5.tar.gz’ s